In [4]:
import pylab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from scipy.spatial import distance
from scipy.optimize import curve_fit
from scipy.optimize import differential_evolution
from mpl_toolkits.axes_grid1 import make_axes_locatable
import string

# Network structure
Variables that define the structure of the network model in the following terms:
- dimensions of the encoded space, 
- number of neurons, 
- receptive fields (RF) centres.

In [15]:
## Tactile area
# Tactile area dimensions (cm)
xt_dim = 10 
yt_dim = 5

# Tactile neurons (count)
Mt = 20
Nt = 10

# Tactile RF centres (cm)
xt = np.arange(1,Mt+1)*0.5 
yt = np.arange(1,Nt+1)*0.5

## Visual area
# Visual area dimensions (cm)
xv_dim = 60
yv_dim = 7

# Visual neurons (count)
Mv = 120
Nv = 14 

# Visual RF centres (cm)
xv = (np.arange(1,Mv+1)*0.5)-2.5
yv = (np.arange(1,Nv+1)*0.5)

# Network fixed parameters
Variables that define the values of the parameters that remain fixed in our study. These parameters are related to: 
- the RF of the unisensory neurons,
- the activation functions of both unisensory and multisensory neurons, 
- the visual and tactile stimuli that is administered during the experiment simulation.  

In [16]:
## Unisensory receptive fields
phit_0 = 1
sigmat_phi = 0.5
phiv_0 = 1
sigmav_phi = 0.5

## Neural activity
# Tactile neurons
ft_min = -0.6 
ft_max = 5
qt_c = 19.43
rt = 0.34

# Visual neurons
fv_min = -0.6 
fv_max = 5 
qv_c = 19.43
rv = 0.34

# Multisensory neuron
fm_min = 0
fm_max = 5
qm_c = 12
rm = 0.6

tau = 20

## External stimuli
# Tactile stimuli
it_0 = 2.5
sigmat_i = 0.3
xt_0 = 5 # cm
yt_0 = 2.5 # cm

# Visual stimuli
iv_0 = 2.5
sigmav_i = 0.3 
yv_0 = 3.5 # cm

# External stimuli functions

In [17]:
def stimt(x,y): 
    """Generate tactile stimuli for the network for a given set of spatial coordinates.

    Args:
        x (number): Spatial coordinate in the x-axis.
        y (number): Spatial coordinate in the y-axis.
        
    Returns:
        I (2D array): Tactile stimuli for the tactile area.
    """ 
        
    # Compute tactile stimulus 
    I = (it_0)*np.exp(- (np.square(xt_0-x)+np.square(yt_0-y))/(2*np.square(sigmat_i)))
    
    return I 


def stimv(x,y,xv_0):
    """Generate visual stimuli for the network.

    Args:
        x (number): Spatial coordinate in the x-axis. 
        y (number): Spatial coordinate in the y-axis.
        xv_0 (number): Central point of the visual stimulus in the x-axis (cm).
        
    Returns:
        I (2D array): Visual stimuli for the visual area.
    """ 
        
    # Compute visual stimulus
    I = (iv_0)*np.exp(- (np.square(xv_0-x)+np.square(yv_0-y))/(2*np.square(sigmav_i)))
    
    return I 

# Receptive fields functions

In [18]:
def phit(x,y):
    """Obtain the RF of the tactile neurons for a given set of spatial coordinates.

    Args:
        x (number): Spatial coordinate in the x-axis (cm). 
        y (number): Spatial coordinate in the y-axis (cm).
        
    Returns:
        phi (2D array): RF of the tactile area for the given spatial coordinates.
    """ 
    
    # Build the RF matrix
    phi = np.zeros((Mt,Nt))
    
    # Compute the RF for the given spatial coordinates
    for i in range(Mt):
        for j in range(Nt):
            phi[i][j] = phit_0*np.exp(-((np.square(xt[i]-x)+np.square(yt[j]-y))/(2*np.square(sigmat_phi))))
    
    return phi

def phiv(x,y):
    """Obtain the RF of the visual neurons for a given set of spatial coordinates.

    Args:
        x (number): Spatial coordinate in the x-axis (cm). 
        y (number): Spatial coordinate in the y-axis (cm).
        
    Returns:
        phi (2D array): RF of the visual area for the given spatial coordinates.
    """ 
    
    # Build the RF matrix
    phi = np.zeros((Mv,Nv))
    
    # Compute the RF for the given spatial coordinates
    for i in range(Mv):
        for j in range(Nv):
            phi[i][j] = phiv_0*np.exp(-((np.square(xv[i]-x)+np.square(yv[j]-y))/(2*np.square(sigmav_phi))))
            
    return phi

def phi_computation():
    """Compute the RF of the unisensory areas for the discreteised spatial coordinates 
        in the visual and tactile spaces.
        
    Returns:
        phi_t (4D array): RF of the tactile area for the discretised spatial coordinates in the tactile space.
        phi_v (4D array): RF of the visual area for the discretised spatial coordinates in the visual space.
    """ 
    
    # Define the step size (cm) for the integral computation 
    dif = 0.2
    
    # Calculate the discretised spatial coordinates  
    xt_i = np.arange(0,xt_dim+dif,dif)
    yt_n = np.arange(0,yt_dim+dif,dif)
    xv_i = np.arange(0,xv_dim+dif,dif)
    yv_n = np.arange(0,yv_dim+dif,dif)

    # Build the RF matrices
    phi_t = np.zeros((Mt,Nt,len(xt_i),len(yt_n)))        
    phi_v = np.zeros((Mv,Nv,len(xv_i),len(yv_n)))
    
    # Compute the RF for the discretised spatial coordinates
    for k in range(len(xt_i)):
        for l in range(len(yt_n)):
            phi_t[:,:,k,l] = phit(xt_i[k],yt_n[l])
            
    for k in range(len(xv_i)):
        for l in range(len(yv_n)):
            phi_v[:,:,k,l] = phiv(xv_i[k],yv_n[l])
    
    return phi_t, phi_v 

### Receptive fields computation
"""Computation of the RF of the unisensory areas for the discreteised spatial coordinates 
        in the visual and tactile spaces using a step size of 0.2 cm. We compute this outside 
        the stimulus functions PHIv and PHIt to reduce the running time of the model. 
""" 
phi_t, phi_v = phi_computation()

# Synapses functions

In [19]:
def Lw(Lt_ex,Lt_in,sigmat_ex,sigmat_in,Lv_ex,Lv_in,sigmav_ex,sigmav_in):
    """Obtain the recurrent synaptic weights of the unisensory neurons.

    Args:
        Lt_ex (number): Amplitude of excitatory synapses in the tactile area. 
        Lt_in (number): Amplitude of inhibitory synapses in the tactile area.
        sigmat_ex (number): Extension of excitatory synapses (cm) in the tactile area. 
        sigmat_in (number): Extension of inhibitory synapses (cm) in the tactile area.
        
        Lv_ex (number): Amplitude of excitatory synapses in the visual area. 
        Lv_in (number): Amplitude of inhibitory synapses in the visual area.
        sigmav_ex (number): Extension of excitatory synapses (cm) in the visual area.
        sigmav_in (number): Extension of inhibitory synapses (cm) in the visual area.
        
        
    Returns:
        Lt (2D array): Recurrent synaptic weights of the tactile area.
        Lv (2D array): Recurrent synaptic weights of the visual area.
    """ 
    
    # Build the recurrent connectivity matrix of the tactile area.
    Lt = np.zeros((Mt*Nt,Mt*Nt))

    # Compute the recurrent connectivity matrix of the tactile area.
    for i in range(Mt*Nt):
        for j in range(Mt*Nt):
            if i==j: 
                Lt[i,j] = 0
            else:
                Dtx = xt[np.floor_divide(i,Nt)] - xt[np.floor_divide(j,Nt)]
                Dty = yt[np.remainder(i,Nt)] - yt[np.remainder(j,Nt)]
                Lt[i,j] = Lt_ex*np.exp(- (np.square(Dtx)+np.square(Dty))/(2*np.square(sigmat_ex)))-Lt_in*np.exp(- (np.square(Dtx)+np.square(Dty))/(2*np.square(sigmat_in)))

    # Build the recurrent synapses matrix of the visual area.
    Lv = np.zeros((Mv*Nv,Mv*Nv))

    # Compute the recurrent synapses matrix of the visual area.
    for i in range(Mv*Nv):
        for j in range(Mv*Nv):
            if i==j: 
                Lv[i,j] = 0
            else:
                Dvx = xv[np.floor_divide(i,Nv)] - xv[np.floor_divide(j,Nv)]
                Dvy = yv[np.remainder(i,Nv)] - yv[np.remainder(j,Nv)] #before was remainder-1
                Lv[i,j] = Lv_ex*np.exp(- (np.square(Dvx)+np.square(Dvy))/(2*np.square(sigmav_ex)))-Lv_in*np.exp(- (np.square(Dvx)+np.square(Dvy))/(2*np.square(sigmav_in)))
    
    return Lt,Lv


def Fw (Wt_0,Wv_0,Bt_0,Bv_0):
    """Feedforward and feedback synaptic weights of the unisensory neurons.

    Args:
        Wt_0 (number): Maximum value of the feedforward synapses in the tactile area.
        Wv_0 (number): Maximum value of the feedforward synapses in the visual area.
        
        Bt_0 (number): Maximum value of the feedback synapses in the tactile area. 
        Bv_0 (number): Maximum value of the feedback synapses in the visual area.
           
    Returns:
        Wt (2D array): Feedforward synaptic weights in the tactile area.
        Wv (2D array): Feedforward synaptic weights in the visual area.
        Bt (2D array): Feedback synaptic weights in the tactile area.
        Bv (2D array): Feedback synaptic weights in the visual area.
    """ 
    
    # Exponential decay parameters 
    k1 = 10 # cm
    k2 = 700 # cm
    lim = 20 # cm 
    alpha = 0.9
    
    # Build the feedforward and feedback synapses matrices
    Bv = np.zeros((Mv,Nv))
    Wv = np.zeros((Mv,Nv))
    Bt = np.ones((Mt,Nt))*Bt_0
    Wt = np.ones((Mt,Nt))*Wt_0

    # Compute the feedforward and feedback synapses in the visual area
    for i in range(Mv):
        for j in range(Nv):
            if (xv[i]<lim):
                D = 0
            else: 
                D = distance.euclidean((xv[i],yv[j]),(lim,yv[j]))              
            Bv[i,j] = alpha*Bv_0*np.exp(- D/k1)+(1-alpha)*Bv_0*np.exp(- D/k2)
            Wv[i,j] = alpha*Wv_0*np.exp(- D/k1)+(1-alpha)*Wv_0*np.exp(- D/k2)
            
    return Wt,Wv,Bt,Bv

# Neural input functions

In [20]:
## Unisensory
def PHIt():
    """Compute the external input of the tactile area as the convolution
        between the external stimulus and the RF of the tactile area.

    Returns:
        PHI (2D array): External input of the tactile area.
    """ 
    
    # Define the step size (cm) for the integral computation 
    dif = 0.2
    
    # Calculate the discretised spatial coordinates of the tactile space
    xt_i = np.arange(0,xt_dim+dif,dif)
    yt_n = np.arange(0,yt_dim+dif,dif)
    
    # Build the external input matrix
    PHI = np.zeros((Mt,Nt,len(xt_i),len(yt_n)))        
    
    # Compute the external input using the histogram rule
    for k in range(len(xt_i)):
        for l in range(len(yt_n)):
            PHI[:,:,k,l] = np.multiply(phi_t[:,:,k,l],stimt(xt_i[k],yt_n[l]))
    PHI = np.sum(PHI,axis=3)
    PHI = np.sum(PHI,axis=2)
    
    return PHI    


def PHIv(xv_0):
    """Compute the external input of the visual area as the convolution
        between the external stimulus and the RF of the tactile area.

    Args:
        xv_0 (number): Central point of the visual stimulus in the x-axis (cm).
        
    Returns:
        PHI (2D array): External input of the visual area.
    """ 
    
    # Define the step size (cm) for the integral computation 
    dif = 0.2
    
    # Calculate the discretised spatial coordinates of the visual space
    xv_i = np.arange(0,xv_dim+dif,dif)
    yv_n = np.arange(0,yv_dim+dif,dif)
    
    # Build the external input matrix
    PHI = np.zeros((Mv,Nv,len(xv_i),len(yv_n)))
    
    # Compute the external visual input using the histogram rule 
    # If the visual stimuli is not delivered (xv_0 = 'uni') returns a zero matrix.
    if xv_0 == 'uni':
        PHI = PHI
    else:
        for k in range(len(xv_i)):
            for l in range(len(yv_n)):
                PHI[:,:,k,l] = np.multiply(phi_v[:,:,k,l],stimv(xv_i[k],yv_n[l],xv_0))
    PHI = np.sum(PHI,axis=3)
    PHI = np.sum(PHI,axis=2)
        
    return PHI
    
    
## Lateral
def LIt(z):
    """Compute the lateral input of the tactile area as the product
        between neural activity and recurrent synaptic weights. 

    Args:
        z (2D array): A matrix of shape (MtxNt) with the activity of tactile neurons. 
        
    Returns:
        LI (2D array): Lateral input of the tactile area.
    """ 
    
    # Build the lateral input matrix
    LI = np.zeros(Mt*Nt)
    
    # Compute the lateral input
    z = np.reshape(z,(1,Mt*Nt))
    for i in range(Mt*Nt):
            LI[i] = np.sum(np.multiply(Lt[i,:],z[0,:])) 
    LI = np.reshape(LI,(Mt,Nt))
    
    return LI


def LIv(z): 
    """Compute the lateral input of the visual area as the product
        between neural activity and recurrent synaptic weights. 

    Args:
        z (2D array): A matrix of shape (MvxNv) with the activity of visual neurons. 
        
    Returns:
        LI (2D array): Lateral input of the visual area.
    """ 
    
    # Build the lateral input matrix
    LI = np.zeros(Mv*Nv)
    
    # Compute the lateral input
    z = np.reshape(z,(1,Mv*Nv))
    for i in range(Mv*Nv):
            LI[i] = np.sum(np.multiply(Lv[i,:],z[0,:])) 
    LI = np.reshape(LI,(Mv,Nv))
    
    return LI


## Feedback
def bt(z):
    """Compute the feeback input of the tactile area as the product
        between neural activity and feedback synaptic weights. 

    Args:
        z (number): Activity of the multisensory neuron. 
        
    Returns:
        bt (2D array): Feedback input of the tactile area.
    """ 
    
    # Compute the feedback input
    bt = np.multiply(Bt,z)
    
    return bt


def bv(z):
    """Compute the feedback input of the visual area as the product
        between neural activity and feedback synaptic weights. 

    Args:
        z (number): Activity of the multisensory neuron. 
        
    Returns:
        bv (2D array): Feedback input of the visual area.
    """ 
    
    # Compute the feedback input
    bv = np.multiply(Bv,z)
    
    return bv

# Neural activity functions

In [3]:
def psit(qt,b): 
    """Compute the activity of the tactile neurons from its overall input. 
    
    Args:
        qt (2D array): State variable of tactile neurons after receiving input.
        b (number): Bias term introduced to generate E/I imbalance
        
    Returns:
        psi (2D array): Neural activity of the tactile area.
    """ 
    
    # Build the activity matrix
    psi = np.zeros((Mt,Nt))
    
    # Compute the neural activity
    for i in range(Mt):
        for j in range(Nt):
            psi[i,j] = (ft_min+ft_max*np.exp((qt[i,j]-qt_c)*rt+b))/(1+np.exp((qt[i,j]-qt_c)*rt+b))
    
    return psi


def psiv(qv,b): 
    """Compute the activity of the visual neurons from its overall input. 
    
    Args:
        qv (2D array): State variable of visual neurons after receiving input.
        b (number): Bias term introduced to generate E/I imbalance
        
    Returns:
        psi (2D array): Neural activity of the visual area.
    """ 
    
    # Build the activity matrix
    psi = np.zeros((Mv,Nv))
    
    # Compute the neural activity
    for i in range(Mv):
        for j in range(Nv):
            psi[i,j] = (fv_min+fv_max*np.exp((qv[i,j]-qv_c)*rv+b))/(1+np.exp((qv[i,j]-qv_c)*rv+b))
            
    return psi


def psim(qm):
    """Compute the activity of the multisensory neuron from its overall input. 
    
    Args:
        qm (number): State variable of the multisensory neuron after receiving input.
        
    Returns:
        psi (number): Neural activity of the multisensory area.
    """ 
    
    # Compute the neural activity
    psi = (fm_min+fm_max*np.exp((qm-qm_c)*rm))/(1+np.exp((qm-qm_c)*rm))
    
    return psi

# Visuotactile experiment function

In [24]:
def experimentrun(v_distances,time,b):
    """Compute the multisensory facilitation effect for the given distance points.
    
    Args:
        v_distances (1D array): Vector with the distances at which the visual stimuli is delivered (cm).
        time (number): Running time of the experiment (ms)
        b (number): E/I imbalance introduced to the network
        
    Returns:
        multi_RTs (1D array): Vector with the multisensory facilitation RTs at the given distance points (ms). 
        ZMs (2D array): Matrix with the activity of the multisensory neuron during the experiment at each distance point.
        ZTs (3D array): Array with the final activity of the tactile neurons at each distance point.
        ZVs (3D array): Array with the final activity of the visual neurons at each distance point.
    """ 
    
    # Include a unisensory measurement in the experiment
    v_distances = np.insert(v_distances.astype('object') , 0, 'uni')
    
    # Network computing parameters
    dt = 0.4
    dtau = dt/tau
    
    # Experiment parameters
    timesteps = int(time/dt)
    ndist = len(v_distances)
    
    # Build matrices to register network activity during the experiment
    RTs = np.zeros(ndist)
    ZTs = np.zeros((Mt,Nt,ndist))
    ZVs = np.zeros((Mv,Nv,ndist))
    
    qt = np.zeros((Mt,Nt,timesteps+1,ndist))
    ut = np.zeros((Mt,Nt,timesteps+1,ndist))
    zt = np.zeros((Mt,Nt,timesteps+1,ndist))
    pt = np.zeros((Mt,Nt,timesteps+1,ndist))

    qv = np.zeros((Mv,Nv,timesteps+1,ndist))
    uv = np.zeros((Mv,Nv,timesteps+1,ndist))
    zv = np.zeros((Mv,Nv,timesteps+1,ndist))
    pv = np.zeros((Mv,Nv,timesteps+1,ndist))

    qm = np.zeros((timesteps+1,ndist))
    um = np.zeros((timesteps+1,ndist))
    zm = np.zeros((timesteps+1,ndist))
    pm = np.zeros((timesteps+1,ndist))
    
    rt = np.zeros((timesteps+1,ndist))
    
    # Generate a tactile stimulus
    ti = PHIt()
    
    # Run the experiment
    for d in range(ndist):
        xv_0 = v_distances[d] # Defines how far the light is presented.     
        vi = PHIv(xv_0) # Generates a visual stimulus.
        
        for i in range(timesteps):    
            # Tactile activity
            ut[:,:,i+1,d] = ti+LIt(zt[:,:,i,d])+bt(zm[i,d])
            qt[:,:,i+1,d] = qt[:,:,i,d] + dtau*(-qt[:,:,i,d]+ut[:,:,i,d])
            pt[:,:,i+1,d] = psit(qt[:,:,i,d],b)
            zt[:,:,i+1,d] = pt[:,:,i,d]*np.heaviside(pt[:,:,i,d],0)

            # Visual activity
            uv[:,:,i+1,d] = vi+LIv(zv[:,:,i,d])+bv(zm[i,d]) 
            qv[:,:,i+1,d] = qv[:,:,i,d] + dtau*(-qv[:,:,i,d]+uv[:,:,i,d])
            pv[:,:,i+1,d] = psiv(qv[:,:,i,d],b)
            zv[:,:,i+1,d] = pv[:,:,i,d]*np.heaviside(pv[:,:,i,d],0)

            # Multisensory activity
            um[i+1,d] = np.sum(np.multiply(Wt,zt[:,:,i,d])) + np.sum(np.multiply(Wv,zv[:,:,i,d]))
            qm[i+1,d] = qm[i,d] + dtau*(-qm[i,d]+um[i,d])
            pm[i+1,d] = psim(qm[i,d])
            zm[i+1,d] = pm[i,d]*np.heaviside(pm[i,d],0)
            
            # Network response
            rt[i+1,d] = np.any(zt[:,:,i,d]>(0.9*ft_max))

        # Computes RT
        RTs[d] = np.argmax(rt[:,d])*dt
        
        # Saves relevant network activity
        ZMs = zm        
        ZTs[:,:,d] = zt[:,:,timesteps,d]
        ZVs[:,:,d] = zv[:,:,timesteps,d]
    
    # Calculate the multisensory facilitation effect
    multiRTs = RTs[1:ndist] - RTs[0]
    
    return multiRTs,ZMs,ZTs,ZVs

# Sigmoid fitting functions

In [25]:
def RTsig(x,cp,h):
    """Compute sigmoidal RT for the given distance points. 
    
    Args:
        x (1D array): Distance points (cm).
        cp (number): Central point of the sigmoid.
        h (number): Slope parameter of the sigmoid (slope = 1/h). 
        
    Returns:
        sig (1D array): Vector with RT for the given distance points. 
    """ 
    # Obtains the upper and lower bounds from global environment
    global ymin
    global ymax
    
    # Compute the sigmoid
    sig = (ymin+ymax*np.exp((x-cp)/h)) / (1+np.exp((x-cp)/h))
    
    return sig


def sigfit(x,y):
    """Fit the RT data to a sigmoidal function.
    
    Args:
        x (1D array): Distance points (cm).
        y (1D array) : RT data to be fitted.
        
    Returns:
        dc (number): Central point of the sigmoid.
        h (number): Slope parameter of the sigmoid (slope = 1/h). 
    """ 
    
    # Obtains the upper and lower bounds from global environment
    global ymin
    global ymax
    
    # Defines starting points and boundaries for the fitting
    k_0 = (ymax - ymin)/(x[-1]-x[0])
    initial_slope = (ymax - ymin)/(4*k_0)
    middle_x = np.max(x)/2
    init_guess = [middle_x, initial_slope]
    boundaries = ([0,float('-inf')],[float('inf'),float('inf')])
    
    # Fits the data
    popt, pcov = curve_fit(RTsig,x,y,p0=init_guess,method='trf',ftol=1e-8,xtol=1e-8,maxfev=10000,bounds=boundaries)
    sigpar = np.asarray(popt)
    dc = sigpar[0]
    h = sigpar[1]
    
    return dc,h